In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src')
import xgboost as xgb
import pandas as pd
from features.UserJoin import UserJoin
from features.UserJoin import submit, diff, plt_month, plt_day
import config as C
import matplotlib.pyplot as plt
import plotly.express as px

/opt/conda/envs/miner/lib/python3.8/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
# data
ds = UserJoin()

In [4]:
ds.train

,ELEC_TYPE_NAME,VOLT_NAME,RUN_CAP,label,kwh,kwh_cal,kwh_pap_r2,kwh_pap_r3,kwh_pap_r4,pr2,...,pq_z,pp,pf,pg,p_f,p_g,f_g,monthcv,ELEC_TYPE_NAME_CODE,VOLT_NAME_CODE
id,,,,,,,,,,,,,,,,,,,,,
329465731,乡村居民生活用电,交流380V,39.0,0,16.818148,16.164545,11.624141,0.000000,4.540404,0.627842,...,637.818182,-0.000020,0.709953,0.290067,0.709973,0.290087,0.419886,1.078650,0,2
329478041,城镇居民生活用电,交流380V,25.0,0,38.505926,38.485888,28.803832,0.000000,9.682056,0.741571,...,1307.818182,0.000071,0.743194,0.256735,0.743123,0.256664,0.486459,1.135459,3,2
329478823,城镇居民生活用电,交流380V,30.0,0,280.084815,277.780374,98.275981,103.011028,76.493364,0.345456,...,8728.000000,0.370042,0.358350,0.271608,0.026347,0.098433,0.086742,0.170947,3,2
329479127,城镇居民生活用电,交流380V,39.0,0,30.249259,30.249444,20.300185,0.000000,9.949259,0.662193,...,1066.227273,0.004394,0.647117,0.348488,0.642723,0.344094,0.298629,0.973298,3,2
329480461,城镇居民生活用电,交流380V,39.0,0,51.833519,51.833333,37.146389,0.000000,14.686944,0.710962,...,1689.045455,0.000014,0.701369,0.298617,0.701356,0.298603,0.402752,1.060835,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2848761050,城镇居民生活用电,交流380V,60.0,test,8.844444,8.838889,4.498148,2.242593,2.098148,0.464905,...,375.000000,0.243862,0.545562,0.210576,0.301700,0.042318,0.334985,0.555885,3,2
2849175989,乡村居民生活用电,交流380V,36.0,test,7.831389,7.831574,5.783056,0.000000,2.048519,0.723773,...,241.909091,0.000315,0.764133,0.235552,0.763818,0.235237,0.528582,1.180742,0,2
2850017469,乡村居民生活用电,交流380V,120.0,test,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,2


In [15]:
t= ds.month.join(ds.month.groupby('id')[['pp', 'pf', 'pg']].mean(), rsuffix='mean')
t.ppmean = t.pp - t.ppmean
t.pfmean = t.pf - t.pfmean
t.pgmean = t.pg - t.pgmean

In [16]:
t

,ym,pq_f,pq_g,pq_p,pq_z,pf,pg,pp,,ELEC_TYPE_NAME,...,TMP_DATE,IS_FLAG,label,p_f,p_g,f_g,monthcv,ppmean,pfmean,pgmean
id,,,,,,,,,,,,,,,,,,,,,
179404030,2020-01-01,487,215,-1,701,0.694722,0.306705,-0.001427,NaN,城镇居民生活用电,...,0.0,NaN,test,0.696148,0.308131,0.388017,1.046512,-0.001078,0.115216,-0.114139
179404030,2020-02-01,736,416,0,1152,0.638889,0.361111,0.000000,NaN,城镇居民生活用电,...,0.0,NaN,test,0.638889,0.361111,0.277778,0.961047,0.000349,0.059383,-0.059732
179404030,2020-03-01,699,477,1,1177,0.593883,0.405268,0.000850,NaN,城镇居民生活用电,...,0.0,NaN,test,0.593033,0.404418,0.188615,0.908970,0.001199,0.014377,-0.015576
179404030,2020-04-01,1122,688,-1,1809,0.620232,0.380321,-0.000553,NaN,城镇居民生活用电,...,0.0,NaN,test,0.620785,0.380873,0.239912,0.939145,-0.000204,0.040727,-0.040523
179404030,2020-05-01,564,452,0,1016,0.555118,0.444882,0.000000,NaN,城镇居民生活用电,...,0.0,NaN,test,0.555118,0.444882,0.110236,0.881670,0.000349,-0.024387,0.024038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2880712108,2021-06-01,474,675,0,1149,0.412533,0.587467,0.000000,NaN,乡村居民生活用电,...,0.0,NaN,test,0.412533,0.587467,0.174935,0.904906,-0.000063,0.087698,-0.042180
2880712108,2021-07-01,342,881,0,1223,0.279640,0.720360,0.000000,NaN,乡村居民生活用电,...,0.0,NaN,test,0.279640,0.720360,0.440720,1.089507,-0.000063,-0.045195,0.090712
2880712108,2021-08-01,293,772,0,1065,0.275117,0.724883,0.000000,NaN,乡村居民生活用电,...,0.0,NaN,test,0.275117,0.724883,0.449765,1.097793,-0.000063,-0.049718,0.095235
